In [1]:
import numpy as np
# import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

In [2]:
import os
for dirname, _, filenames in os.walk('../Dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated2.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1-rotated3.jpg
../Dataset\archive (6)\train\fractured\10-rotated1.jpg
../Dataset\archive (6)\train\fractured\10-rotated2-rotated1-rotated1.jpg
../Dataset\archive (6)\

In [3]:
train_path= '../Dataset/archive (6)/train'
test_path='../Dataset/archive (6)/val'

In [4]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
    )
val_datagen= image.ImageDataGenerator(    
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
    )

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary'
    )

validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary'
    )

Found 8863 images belonging to 2 classes.
Found 600 images belonging to 2 classes.


In [10]:
base_model2 = tf.keras.applications.EfficientNetV2B0(
                    include_top=False,
                    weights="imagenet",
                    input_tensor=None,
                    input_shape=(224,224,3),
                    pooling=None,
                    classes=1000,
                    classifier_activation="softmax",
                    include_preprocessing=True,
                )

for layer in base_model2.layers:
    layer.trainable=False

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=[224, 224, 3]),
    base_model2,
    tf.keras.layers.GaussianNoise(0.25),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GaussianNoise(0.2),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Function  (None, 7, 7, 1280)       5919312   
 al)                                                             
                                                                 
 gaussian_noise_4 (GaussianN  (None, 7, 7, 1280)       0         
 oise)                                                           
                                                                 
 global_average_pooling2d_3   (None, 1280)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_6 (Dense)             (None, 512)               655872    
                                                                 
 batch_normalization_2 (Batc  (None, 512)              2048      
 hNormalization)                                      

In [11]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

lrp = ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=2)

checkpoint = ModelCheckpoint(filepath='model_checkpoint.h5', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max', save_weights_only=True)
callbacks = [checkpoint, lrp]

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    steps_per_epoch=50,
    callbacks=callbacks
)

Epoch 1/10
50/50 [==============================] - ETA: 0s - loss: 0.7778 - accuracy: 0.6050
Epoch 1: val_accuracy improved from -inf to 0.49667, saving model to model_checkpoint.h5
50/50 [==============================] - 48s 791ms/step - loss: 0.7778 - accuracy: 0.6050 - val_loss: 0.8500 - val_accuracy: 0.4967 - lr: 0.0010
Epoch 2/10
50/50 [==============================] - ETA: 0s - loss: 0.7825 - accuracy: 0.6350
Epoch 2: val_accuracy improved from 0.49667 to 0.52167, saving model to model_checkpoint.h5
50/50 [==============================] - 35s 714ms/step - loss: 0.7825 - accuracy: 0.6350 - val_loss: 1.0219 - val_accuracy: 0.5217 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - ETA: 0s - loss: 0.7394 - accuracy: 0.6250
Epoch 3: val_accuracy improved from 0.52167 to 0.60333, saving model to model_checkpoint.h5
50/50 [==============================] - 33s 674ms/step - loss: 0.7394 - accuracy: 0.6250 - val_loss: 1.0625 - val_accuracy: 0.6033 - lr: 0.0010
Epoch 4/10

In [13]:
model.evaluate(train_generator)

 407/2216 [====>.........................] - ETA: 6:48 - loss: 0.4501 - accuracy: 0.7783

KeyboardInterrupt: 

In [14]:
model.evaluate(validation_generator)

150/150 [==============================] - 27s 176ms/step - loss: 0.7346 - accuracy: 0.6783


[0.7345871329307556, 0.6783333420753479]